In [46]:
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

################# Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance

print("Done uploading libraries")

Done uploading libraries


In [47]:
############################## UCI Italian dataset #######################################################
## 2 years and single terrain
aqi_df = pd.read_csv('AQI_datasets/UCI_AQI/AirQualityUCI.csv', sep=',', delimiter=";", decimal=",", index_col = None, header=0)

aqi_df.head(20)
# aqi_df.shape
aqi_df.columns

## Predictors: T,Ah,Rh, NMHC_GT, NOx_GT, CO_GT, C6H6_GT ,Target: O3 

Index(['Date', 'Time', 'CO_GT', 'PT08_S1_CO', 'NMHC_GT', 'C6H6_GT',
       'PT08_S2_NMHC', 'NOx_GT', 'PT08_S3_NOx', 'NO2_GT', 'PT08_S4_NO2',
       'PT08_S5_O3', 'T', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [48]:
def remove_outlier(col):
    aqi_df[col] = aqi_df.groupby('Date')[col].transform(lambda x: x.fillna(x.mean()))


#drop end rows with NaN values
aqi_df.dropna(how = 'all', inplace = True)

drop_unamed = ['Unnamed: 15', 'Unnamed: 16']
aqi_df = aqi_df.drop(drop_unamed, axis = 1)
aqi_df.columns
# aqi_df.shape
# aqi_df

#Observing data statistics
aqi_df.describe()

#Drop unwanted columns
drop_uw = ['Time', 'PT08_S1_CO', 'PT08_S2_NMHC', 'PT08_S5_O3', 'PT08_S3_NOx', 'PT08_S4_NO2']
aqi_df = aqi_df.drop(drop_uw, axis = 1)
aqi_df

#Replace the -200 values seen in the dataset with 
aqi_df.replace(to_replace = -200, value = np.NaN, inplace = True)

#Replace the NaN values with the column mean
col_list = aqi_df.columns[1:]
for i in col_list:
    remove_outlier(i)

aqi_df.fillna(method ='ffill', inplace= True)
aqi_df.dropna(axis = 0)
aqi_df

#Convert 'Date' column to datetime and then seperate out year and month into different columns.
aqi_df.Date = pd.to_datetime(aqi_df.Date)
aqi_df['Year'] = aqi_df['Date'].dt.year
aqi_df['Month'] = aqi_df['Date'].dt.month
drop_date = ['Date']
aqi_df = aqi_df.drop(drop_date, axis = 1)
aqi_df


,CO_GT,NMHC_GT,C6H6_GT,NOx_GT,NO2_GT,T,RH,AH,Year,Month
0,2.6,150.0,11.9,166.0,113.0,13.6,48.9,0.7578,2004,10
1,2.0,112.0,9.4,103.0,92.0,13.3,47.7,0.7255,2004,10
2,2.2,88.0,9.0,131.0,114.0,11.9,54.0,0.7502,2004,10
3,2.2,80.0,9.2,172.0,122.0,11.0,60.0,0.7867,2004,10
4,1.6,51.0,6.5,131.0,116.0,11.2,59.6,0.7888,2004,10
...,...,...,...,...,...,...,...,...,...,...
9352,3.1,275.0,13.5,472.0,190.0,21.9,29.3,0.7568,2005,4
9353,2.4,275.0,11.4,353.0,179.0,24.3,23.7,0.7119,2005,4
9354,2.4,275.0,12.4,293.0,175.0,26.9,18.3,0.6406,2005,4
9355,2.1,275.0,9.5,235.0,156.0,28.3,13.5,0.5139,2005,4


In [49]:
#Split the dataset according to the year.
aqi_df_2004 = aqi_df[aqi_df['Year'] == 2004]
aqi_df_2005 = aqi_df[aqi_df['Year'] == 2005]

aqi_df_2004 = aqi_df_2004.reset_index(drop=True)
aqi_df_2005 = aqi_df_2005.reset_index(drop=True)


#Divide the dataframe into target and the predictors.
target_uci_col = ['NOx_GT']
aqi_df_2004_target = aqi_df_2004[target_uci_col]
aqi_df_2004_target.columns = ['NOx']

aqi_df_2005_target = aqi_df_2005[target_uci_col]
aqi_df_2005_target.columns = ['NOx']

aqi_df_2004_predictors = aqi_df_2004.drop(target_uci_col, axis = 1)
aqi_df_2004_predictors.columns = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH', 'Year', 'Month']

aqi_df_2005_predictors = aqi_df_2005.drop(target_uci_col, axis = 1)
aqi_df_2005_predictors.columns = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH', 'Year', 'Month']

aqi_df_2004_target
aqi_df_2004_predictors

aqi_df_2005_target
aqi_df_2005_predictors


,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month
0,2.134783,275.0,4.2,128.409091,8.2,40.1,0.4375,2005,1
1,1.600000,275.0,8.8,106.000000,5.3,50.7,0.4564,2005,1
2,2.500000,275.0,7.5,129.000000,5.9,50.0,0.4689,2005,1
3,2.700000,275.0,7.6,128.409091,4.9,53.9,0.4693,2005,1
4,1.900000,275.0,5.6,126.000000,4.3,55.3,0.4650,2005,1
...,...,...,...,...,...,...,...,...,...
2242,3.100000,275.0,13.5,190.000000,21.9,29.3,0.7568,2005,4
2243,2.400000,275.0,11.4,179.000000,24.3,23.7,0.7119,2005,4
2244,2.400000,275.0,12.4,175.000000,26.9,18.3,0.6406,2005,4
2245,2.100000,275.0,9.5,156.000000,28.3,13.5,0.5139,2005,4


In [50]:
## Standardize the dataset
columns_uci = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH', 'Year', 'Month']
cols_to_norm = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH']

ss = StandardScaler()
aqi_df_2004_predictors[cols_to_norm] = ss.fit_transform(aqi_df_2004_predictors[cols_to_norm])
aqi_df_2005_predictors[cols_to_norm] = ss.fit_transform(aqi_df_2005_predictors[cols_to_norm])

aqi_df_2004_predictors
# aqi_df_2005_predictors

,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month
0,0.357434,-1.403072,0.167727,0.358847,-0.897718,0.030505,-1.028223,2004,10
1,-0.073637,-1.872833,-0.163670,-0.177882,-0.935168,-0.038507,-1.115824,2004,10
2,0.070053,-2.169524,-0.216693,0.384406,-1.109937,0.323803,-1.048835,2004,10
3,0.070053,-2.268421,-0.190181,0.588874,-1.222288,0.668860,-0.949843,2004,10
4,-0.361018,-2.626923,-0.548090,0.435523,-1.197321,0.645856,-0.944148,2004,10
...,...,...,...,...,...,...,...,...,...
7105,-0.648398,0.142195,-0.707160,0.052145,-1.434508,-0.849390,-2.010003,2004,12
7106,-0.648398,0.142195,-0.707160,0.052145,-1.397057,-0.924153,-2.033598,2004,12
7107,-0.648398,0.142195,-0.680649,0.052145,-1.459475,-0.912651,-2.062075,2004,12
7108,-0.648398,0.142195,-0.667393,0.052145,-1.459475,-0.642356,-1.914537,2004,12


In [51]:
#Splitting dataset into train, test and source.
### Case 1: 0.96
### Case 2: 0.999
X_2005_train, X_2005_test, y_2005_train, y_2005_test = train_test_split(aqi_df_2005_predictors, aqi_df_2005_target, test_size = 0.96, random_state = 1)

X_2004 = aqi_df_2004_predictors
y_2004 = aqi_df_2004_target

X_source = X_2004
X_source['NOx'] = y_2004

X_train = X_2005_train
X_train['NOx'] = y_2005_train

X_test = X_2005_test
X_test['NOx'] = y_2005_test

X_source, X_train.shape, X_test.shape

<ipython-input-51-bce9a2e6f612>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['NOx'] = y_2005_train
<ipython-input-51-bce9a2e6f612>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['NOx'] = y_2005_test


(            CO      NMHC      C6H6       NO2      Temp        RH        AH  \
 0     0.357434 -1.403072  0.167727  0.358847 -0.897718  0.030505 -1.028223   
 1    -0.073637 -1.872833 -0.163670 -0.177882 -0.935168 -0.038507 -1.115824   
 2     0.070053 -2.169524 -0.216693  0.384406 -1.109937  0.323803 -1.048835   
 3     0.070053 -2.268421 -0.190181  0.588874 -1.222288  0.668860 -0.949843   
 4    -0.361018 -2.626923 -0.548090  0.435523 -1.197321  0.645856 -0.944148   
 ...        ...       ...       ...       ...       ...       ...       ...   
 7105 -0.648398  0.142195 -0.707160  0.052145 -1.434508 -0.849390 -2.010003   
 7106 -0.648398  0.142195 -0.707160  0.052145 -1.397057 -0.924153 -2.033598   
 7107 -0.648398  0.142195 -0.680649  0.052145 -1.459475 -0.912651 -2.062075   
 7108 -0.648398  0.142195 -0.667393  0.052145 -1.459475 -0.642356 -1.914537   
 7109 -0.648398  0.142195 -0.799951  0.052145 -1.621760 -0.573345 -1.975559   
 
       Year  Month    NOx  
 0     2004     10  16

In [52]:
######################################## Seeding Technique ###################################################
kmeans = KMeans(n_clusters = 9, random_state=0).fit(X_source)

Alternate_df = X_train.copy()
idxlist = []
Alternate_df_np = Alternate_df.to_numpy()

new_df_list = []


for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in Alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

    print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
    print("Min. distance: ", mindist)
    print("Matrix shape: ", Alternate_df_np.shape)
    new_df_list.append(rowval)
    Alternate_df_np = np.delete(Alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


new_df = pd.DataFrame(np.vstack(new_df_list))
#new_df
new_df.shape
#new_df.dtypes



Row selected:  74
Min. distance:  912.7715872062486
Matrix shape:  (89, 10)
Row selected:  20
Min. distance:  772.3761604547398
Matrix shape:  (88, 10)
Row selected:  41
Min. distance:  504.30971141391166
Matrix shape:  (87, 10)
Row selected:  66
Min. distance:  559.7742067083276
Matrix shape:  (86, 10)
Row selected:  84
Min. distance:  570.8319553379229
Matrix shape:  (85, 10)
Row selected:  67
Min. distance:  1073.926068802342
Matrix shape:  (84, 10)
Row selected:  21
Min. distance:  372.42016652848247
Matrix shape:  (83, 10)
Row selected:  73
Min. distance:  407.154236932868
Matrix shape:  (82, 10)
Row selected:  51
Min. distance:  509.9861664911643
Matrix shape:  (81, 10)


(9, 10)

In [53]:
##################################################### Phase 2: Seeding ################################################

alt_source_df = X_source.copy()
idxlist2 = []
alt_source_df_np = alt_source_df.to_numpy()

final_df_list = []

for row_nm in new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in alt_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

    print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
    print("Min. distance: ", min_dist)
    print("Matrix shape: ", alt_source_df_np.shape)
    final_df_list.append(row_val)
    alt_source_df_np = np.delete(alt_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


final_df = pd.DataFrame(np.vstack(final_df_list), columns= X_source.columns)
final_df
final_df.shape

print("Shape before :",X_source.shape)
X_source = pd.concat([X_source, final_df])
X_source = X_source.drop_duplicates(keep=False)
print("Shape after :",X_source.shape)

Row selected:  7109
Min. distance:  801.0890782673067
Matrix shape:  (7110, 10)
Row selected:  6986
Min. distance:  548.1285851288357
Matrix shape:  (7109, 10)
Row selected:  7107
Min. distance:  698.0173642865282
Matrix shape:  (7108, 10)
Row selected:  7106
Min. distance:  578.0232036782321
Matrix shape:  (7107, 10)
Row selected:  6962
Min. distance:  767.0911383692626
Matrix shape:  (7106, 10)
Row selected:  6961
Min. distance:  533.1280259309375
Matrix shape:  (7105, 10)
Row selected:  6993
Min. distance:  548.1175871471462
Matrix shape:  (7104, 10)
Row selected:  6955
Min. distance:  530.1236340422324
Matrix shape:  (7103, 10)
Row selected:  6991
Min. distance:  578.1257502042014
Matrix shape:  (7102, 10)
Shape before : (7110, 10)
Shape after : (7071, 10)


In [54]:
X_train = pd.concat([X_train, final_df], ignore_index=True)

X_train.to_csv('ActiveSampling/O3_seeded_train.csv',index=False)
X_source.to_csv('ActiveSampling/O3_seeded_source.csv',index=False)

X_train
# X_train.to_csv('ActiveSampling/UCI_NO2_activesampling_train.csv',index=False)
# X_source.to_csv('ActiveSampling/UCI_NO2_activesampling_source.csv',index=False)
# X_test.to_csv('ActiveSampling/UCI_NO2_activesampling_test.csv',index=False)

,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month,NOx
0,-0.236089,0.000000,-0.093770,-0.857417,-0.692434,1.618722,0.377326,2005.0,7.0,284.0
1,-0.753174,0.000000,-0.963184,-0.995832,-0.961145,-0.867393,-1.269811,2005.0,7.0,104.0
2,-0.679304,0.000000,-0.652679,-0.501494,1.564738,-1.575074,-0.319355,2005.0,2.0,161.0
3,-0.753174,0.000000,-0.544002,-1.173794,0.024128,1.391033,1.089171,2005.0,12.0,183.0
4,-0.236089,0.000000,0.828970,-0.264211,0.255453,0.841476,0.950868,2005.0,5.0,266.0
...,...,...,...,...,...,...,...,...,...,...
93,2.943861,0.142195,1.413779,1.917919,-1.184838,1.721283,-0.247952,2004.0,12.0,821.0
94,1.938028,0.142195,0.976335,1.815685,-1.234772,1.583260,-0.407153,2004.0,12.0,592.0
95,-1.151315,0.142195,-1.224139,-0.816846,-1.035036,0.513584,-0.840817,2004.0,12.0,93.0
96,2.440945,0.142195,0.989591,2.020153,-1.172354,0.835637,-0.790643,2004.0,12.0,602.0


In [55]:
X_train = pd.read_csv('ActiveSampling/O3_seeded_train.csv')
X_source = pd.read_csv('ActiveSampling/O3_seeded_source.csv')

X_source['ManDis'] = 0.0

train_df_mean = []
prow = X_train.mean()
train_df_mean = [prow.CO, prow.NMHC, prow.C6H6, prow.NO2, prow.Temp, prow.RH, prow.AH, prow.Year, prow.Month, prow.NOx]

rowidx = 0
for row in X_source.itertuples():
    row_list =[row.CO, row.NMHC, row.C6H6, row.NO2, row.Temp, row.RH, row.AH, row.Year, row.Month, row.NOx]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = train_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)
    
    X_source.loc[rowidx,"ManDis"] = man_dis
    # print(Source_df.loc[rowidx,"ManDis"])
    rowidx = rowidx + 1


X_source_rest = X_source.sort_values('ManDis')
X_source = X_source.head(2000)
X_source_rest = X_source.iloc[2000:]
X_source_rest = X_source_rest.drop(['ManDis'], axis =1)
X_source = X_source.drop(['ManDis'], axis =1)

X_train.to_csv('ActiveSampling/UCI_O3_activesampling_train.csv',index=False)
X_source.to_csv('ActiveSampling/UCI_O3_activesampling_source.csv',index=False)
X_test.to_csv('ActiveSampling/UCI_O3_activesampling_test.csv',index=False)

print("Done !!!!!")
X_source
X_train

Done !!!!!


,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month,NOx
0,-0.236089,0.000000,-0.093770,-0.857417,-0.692434,1.618722,0.377326,2005.0,7.0,284.0
1,-0.753174,0.000000,-0.963184,-0.995832,-0.961145,-0.867393,-1.269811,2005.0,7.0,104.0
2,-0.679304,0.000000,-0.652679,-0.501494,1.564738,-1.575074,-0.319355,2005.0,2.0,161.0
3,-0.753174,0.000000,-0.544002,-1.173794,0.024128,1.391033,1.089171,2005.0,12.0,183.0
4,-0.236089,0.000000,0.828970,-0.264211,0.255453,0.841476,0.950868,2005.0,5.0,266.0
...,...,...,...,...,...,...,...,...,...,...
93,2.943861,0.142195,1.413779,1.917919,-1.184838,1.721283,-0.247952,2004.0,12.0,821.0
94,1.938028,0.142195,0.976335,1.815685,-1.234772,1.583260,-0.407153,2004.0,12.0,592.0
95,-1.151315,0.142195,-1.224139,-0.816846,-1.035036,0.513584,-0.840817,2004.0,12.0,93.0
96,2.440945,0.142195,0.989591,2.020153,-1.172354,0.835637,-0.790643,2004.0,12.0,602.0
